<a href="https://colab.research.google.com/github/djparker29/predictStocks/blob/main/Coding%20Section.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import sys
import sys

In [2]:
# User settings
tickerSymbol = 'TSLA'

# Dates
startDate = '2018-04-01'
endDate = '2021-01-01'

In [3]:
!{sys.executable} -m pip install yfinance

In [4]:
import yfinance as yf

tickerData = yf.Ticker(tickerSymbol)
df = tickerData.history(start=startDate, end=endDate)

In [5]:
import pandas as pd
import numpy as np

date_change = '%y-%m-%d'
df['Date'] = df.index
df['Date']=pd.to_datetime(df['Date'], format=date_change)

In [6]:
!{sys.executable} -m pip install ta
from ta import add_all_ta_features

In [7]:
df=add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True)
df.columns

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_mfi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi',
       'volume_vwap', 'volatility_atr', 'volatility_bbm', 'volatility_bbh',
       'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_

In [8]:
!{sys.executable} -m pip install fastai
import fastai.tabular

In [9]:
fastai.tabular.add_datepart(df, 'Date', drop=True)

/usr/local/lib/python3.6/dist-packages/fastai/tabular/transform.py:63: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  for n in attr: df[prefix + n] = getattr(field.dt, n.lower())


,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,...,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-04-02,51.251999,52.066002,48.917999,50.495998,80570000,0,0.0,2.046396e+05,80570000,0.002540,0.000000e+00,50.000000,0.000000,0.000000,-5.162536e+07,1000.000000,50.493333,0.000000,50.495998,50.495998,50.495998,0.000000,0.000000,0.0,0.0,50.493333,53.641336,47.345331,12.468983,0.500423,0.0,0.0,48.917999,52.066002,50.492001,6.234163,0.501270,0.000000,0.000000,0.000000,...,127.490419,4.0,4.0,0.0,-1.000000,-1.000000,0.0,0.0,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.126995,50.126995,-49.873005,0.000000,50.495998,0.000000,0.000000,0.000000,0.000000,-64.571904,0.000000,0.000000,2018,4,14,2,0,92,False,False,False,False,False,False,1522627200
2018-04-03,53.964001,54.669998,50.897999,53.506001,94222000,0,0.0,3.627489e+07,174792000,0.207532,2.836084e+08,100.000000,9.175580,9.175580,-4.640913e+07,1000.000000,51.857853,0.000000,52.000999,55.011002,48.990997,11.576709,0.750000,0.0,0.0,51.759000,55.219000,48.298999,13.369659,0.752457,0.0,0.0,48.917999,54.669998,51.793999,11.061324,0.797636,0.000000,0.240114,0.048023,...,127.490419,8.0,4.0,4.0,-1.000000,-1.000000,0.0,0.0,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,41.109002,79.763597,64.945296,-20.236403,0.000000,51.773199,0.000000,1.334925,0.266985,1.067940,5.960873,5.789971,5.960873,2018,4,14,3,1,93,False,False,False,False,False,False,1522713600
2018-04-04,50.556000,57.674000,50.400002,57.388000,99483500,0,0.0,1.279354e+08,274275500,0.466449,2.982636e+08,100.000000,9.161656,9.168618,1.283424e+07,1000.000000,53.053411,0.000000,53.796666,59.438956,48.154377,20.976355,0.818251,0.0,0.0,52.890667,57.622000,48.159333,17.890996,0.975271,0.0,0.0,48.917999,57.674000,53.296000,16.276102,0.967337,0.000000,0.735177,0.185454,...,127.490419,12.0,4.0,8.0,-1.000000,52.066002,0.0,1.0,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,68.498223,96.733677,75.541423,-3.266323,0.000000,53.817303,0.000000,2.834009,0.780390,2.053619,7.255261,7.004142,13.648610,2018,4,14,4,2,94,False,False,False,False,False,False,1522800000
2018-04-05,57.868000,61.251999,57.639999,61.144001,95605500,0,0.0,2.178237e+08,369881000,0.588902,3.069537e+08,100.000000,20.435331,12.924189,1.347509e+07,1065.449231,54.852042,0.000000,55.633500,63.656222,47.610778,28.841335,0.843431,0.0,0.0,54.671000,59.122500,50.219500,16.284685,1.227058,1.0,0.0,48.917999,61.251999,55.084999,22.170095,0.991244,0.000000,1.414292,0.431221,...,127.490419,16.0,4.0,12.0,50.400002,52.066002,1.0,0.0,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,74.507055,99.124389,91.873888,-0.875611,0.000000,56.353262,0.000000,3.579195,1.340151,2.239045,6.544923,6.339652,21.086825,2018,4,14,5,3,95,False,False,False,False,False,False,1522886400
2018-04-06,60.200001,61.855999,59.099998,59.860001,67601500,0,0.0,1.875062e+08,302279500,0.428603,2.507032e+08,100.000000,4.207290,10.744

In [10]:
df['Date']=pd.to_datetime(df.index.values, format=date_change)
fastai.tabular.add_cyclic_datepart(df, 'Date', drop=True)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,...,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,weekday_cos,weekday_sin,day_month_cos,day_month_sin,month_year_cos,month_year_sin,day_year_cos,day_year_sin
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-04-02,51.251999,52.066002,48.917999,50.495998,80570000,0,0.0,2.046396e+05,80570000,0.002540,0.000000e+00,50.000000,0.000000,0.000000,-5.162536e+07,1000.000000,50.493333,0.000000,50.495998,50.495998,50.495998,0.000000,0.000000,0.0,0.0,50.493333,53.641336,47.345331,12.468983,0.500423,0.0,0.0,48.917999,52.066002,50.492001,6.234163,0.501270,0.000000,0.000000,0.000000,...,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.126995,50.126995,-49.873005,0.000000,50.495998,0.000000,0.000000,0.000000,0.000000,-64.571904,0.000000,0.000000,2018,4,14,2,0,92,False,False,False,False,False,False,1522627200,1.000000,0.000000,0.978148,0.207912,6.123234e-17,1.0,0.004304,0.999991
2018-04-03,53.964001,54.669998,50.897999,53.506001,94222000,0,0.0,3.627489e+07,174792000,0.207532,2.836084e+08,100.000000,9.175580,9.175580,-4.640913e+07,1000.000000,51.857853,0.000000,52.000999,55.011002,48.990997,11.576709,0.750000,0.0,0.0,51.759000,55.219000,48.298999,13.369659,0.752457,0.0,0.0,48.917999,54.669998,51.793999,11.061324,0.797636,0.000000,0.240114,0.048023,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,41.109002,79.763597,64.945296,-20.236403,0.000000,51.773199,0.000000,1.334925,0.266985,1.067940,5.960873,5.789971,5.960873,2018,4,14,3,1,93,False,False,False,False,False,False,1522713600,0.623490,0.781831,0.913545,0.406737,6.123234e-17,1.0,-0.012910,0.999917
2018-04-04,50.556000,57.674000,50.400002,57.388000,99483500,0,0.0,1.279354e+08,274275500,0.466449,2.982636e+08,100.000000,9.161656,9.168618,1.283424e+07,1000.000000,53.053411,0.000000,53.796666,59.438956,48.154377,20.976355,0.818251,0.0,0.0,52.890667,57.622000,48.159333,17.890996,0.975271,0.0,0.0,48.917999,57.674000,53.296000,16.276102,0.967337,0.000000,0.735177,0.185454,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,68.498223,96.733677,75.541423,-3.266323,0.000000,53.817303,0.000000,2.834009,0.780390,2.053619,7.255261,7.004142,13.648610,2018,4,14,4,2,94,False,False,False,False,False,False,1522800000,-0.222521,0.974928,0.809017,0.587785,6.123234e-17,1.0,-0.030120,0.999546
2018-04-05,57.868000,61.251999,57.639999,61.144001,95605500,0,0.0,2.178237e+08,369881000,0.588902,3.069537e+08,100.000000,20.435331,12.924189,1.347509e+07,1065.449231,54.852042,0.000000,55.633500,63.656222,47.610778,28.841335,0.843431,0.0,0.0,54.671000,59.122500,50.219500,16.284685,1.227058,1.0,0.0,48.917999,61.251999,55.084999,22.170095,0.991244,0.000000,1.414292,0.431221,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,74.507055,99.124389,91.873888,-0.875611,0.000000,56.353262,0.000000,3.579195,1.340151,2.239045,6.544923,6.339652,21.086825,2018,4,14,5,3,95,False,False,False,False,False,False,1522886400,-0.900969,0.433884,0.669131,0.743145,6.123234e-17,1.0,-0.047321,0.998880
2018-04-06,60.200001,61.855999,59.099998,59.860001,67601500,0,0.0,1.875062e+08,302279500,0.428603,2

In [11]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'volume_adi', 'volume_obv', 'volume_cmf',
       ...
       'Is_year_start', 'Elapsed', 'weekday_cos', 'weekday_sin',
       'day_month_cos', 'day_month_sin', 'month_year_cos', 'month_year_sin',
       'day_year_cos', 'day_year_sin'],
      dtype='object', length=111)

In [12]:
# Defining model parameters

# Shifts
shifts = [1, 5, 10]

# Set a training percentage
train_pct = 0.75

# Plotting dimensions
w = 16 #width
h = 6 #height

In [13]:
def createLags(df, lag_size):
  shiftDays = lag_size
  shift = -shiftDays
  df['Close_lag'] = df['Close'].shift(shift)
  return df, shift

In [14]:
# Ensure columns are the right types
def correctColumnsTypes(df):
  # Numbers
  for col in df.columns[1:80]:
    df[col] = df[col].astype('float')
  
  for  col in df.columns[-10:]:
    df[col] = df[col].astype('float')
  # Categories
  for col in df.columns[80:-10]:
    df[col] = df[col].astype('category')

  return df  

In [15]:
## Function to split the data up using train vs. tesing
def splitData(df, train_pct, shift):
  train_pt = int(len(df)*train_pct)

  train = df.iloc[:train_pt, :]
  test = df.iloc[train_pt:, :]

  x_train = train.iloc[:shift, 1: -1]
  y_train = train['Close_lag'][:shift]
  x_test = test.iloc[:shift, 1:-1]
  y_test = test['Close_lag'][:shift]

  return x_train, y_train, x_test, y_test, train, test

In [16]:
!{sys.executable} -m pip install plotly
import plotly.graph_objs as go

In [17]:
# Making visualzatios helper functions
def plotModelResultsPlotly(train, test, pred, tickerSymbol, w, h, shift_days, name):
  D1 = go.Scatter(x=train.index, y=train['Close'], name='Train Actual') #train data
  D2 = go.Scatter(x=test.index, y=test['Close'], name='Test Actual') #test data
  D3 = go.Scatter(x=test.index, y=pred, name='Our Prediction') #prediction

  line = {
    'data': [D1, D2, D3],
    'layout': {
        'xaxis': {'title': 'Date'},
        'yaxis': {'title': '$'},
        'title': name + ' - ' + tickerSymbol + ' - ' + str(shift_days)
    }
  }

  fig = go.Figure(line)
  fig.show()

In [18]:
!{sys.executable} -m pip install sklearn
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [19]:
# Regression
def linearRegressionFunction(x_train, y_train, x_test, y_test):
  lr = LinearRegression()
  lr.fit(x_train, y_train)
  lr_pred = lr.predict(x_test)

  lr_MSE = mean_squared_error(y_test, lr_pred)
  lr_R2 = lr.score(x_test, y_test)

  print('Mean Squared Error: {}'.format(lr_MSE))
  print('R2: {}'.format(lr_R2))

  return lr_pred

In [20]:
# Regression
def artificialNeuralNetworkFunction(x_train, y_train, x_test, y_test):
  
  # Scale the data
  scaler = StandardScaler()
  scaler.fit(x_train)
  x_train_scaled = scaler.transform(x_train)
  x_test_scaled = scaler.transform(x_test)
  
  MLP = MLPRegressor(activation='identity', learning_rate='adaptive', max_iter=1000, random_state=1)
  MLP.fit(x_train_scaled, y_train)
  MLP_pred = MLP.predict(x_test_scaled)

  MLP_MSE = mean_squared_error(y_test, MLP_pred)
  MLP_R2 = MLP.score(x_test_scaled, y_test)

  print('Mean Squared Error: {}'.format(MLP_MSE))
  print('R2: {}'.format(MLP_R2))

  return MLP_pred

In [21]:
# Calculating the profit
def calcProfit(test_df, pred, j):
  pd.set_option('mode.chained_assignment', None)

  test_df['pred'] = np.nan
  test_df['pred'].iloc[:-j] = pred
  test_df['change'] = test_df['Close_lag'] - test_df['Close']
  test_df['change_pred'] = test_df['pred'] - test_df['Close']
  test_df['MadeMoney'] = np.where(test_df['change_pred']/test_df['change']>0, 1, -1)
  test_df['profit'] = np.abs(test['change_pred']) * test_df['MadeMoney']
  profit_dollars = test_df['profit'].sum()
  print('Would have made around $' + str(round(profit_dollars, 1)))
  profit_days = len(test_df[test_df['MadeMoney'] == 1])
  print('Percentage of good trading days: ' + str( round(profit_days/(len(test_df)-j), 2)))


In [22]:
# Here is the algorithm

for j in shifts:
  print(str(j) + 'days out prediction:')
  print('-------------------------')

  # Create the lags
  df_lag, shift = createLags(df, j)

  # Ensure column types
  x_train, y_train, x_test, y_test, train, test = splitData(df_lag, train_pct, shift)

  print('Preprocessing is done!')

  # Linear Regression
  print('Linear Regression')
  lr_pred = linearRegressionFunction(x_train, y_train, x_test, y_test)
  calcProfit(test, lr_pred, j)
  plotModelResultsPlotly(train, test, lr_pred, tickerSymbol, w, h, j, 'Linear Regression')


  print('---------------------')

  # ANN
  print('ANN')
  MLP_pred = artificialNeuralNetworkFunction(x_train, y_train, x_test, y_test)
  calcProfit(test, MLP_pred, j)
  plotModelResultsPlotly(train, test, MLP_pred, tickerSymbol, w, h, j, 'ANN')

1days out prediction:
-------------------------
Preprocessing is done!
Linear Regression
Mean Squared Error: 11521.56537602946
R2: 0.4971182600249976
Would have made around $-2298.0
Percentage of good trading days: 0.43


---------------------
ANN
Mean Squared Error: 1849.1018371748871
R2: 0.919292255963405
Would have made around $-790.6
Percentage of good trading days: 0.43


5days out prediction:
-------------------------
Preprocessing is done!
Linear Regression
Mean Squared Error: 93697.30801129245
R2: -3.1927103583787932
Would have made around $-11889.8
Percentage of good trading days: 0.31


---------------------
ANN
Mean Squared Error: 20560.276062290133
R2: 0.07998122627917859
Would have made around $-2154.0
Percentage of good trading days: 0.36


10days out prediction:
-------------------------
Preprocessing is done!
Linear Regression
Mean Squared Error: 56699.93417850455
R2: -1.6274480025017013
Would have made around $-9311.0
Percentage of good trading days: 0.27


---------------------
ANN
Mean Squared Error: 14110.337929585148
R2: 0.3461336464519629
Would have made around $-4269.7
Percentage of good trading days: 0.3
